#model import requirements

In [ ]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
device = "cpu"

In [ ]:
class RNN(nn.Module):
  def __init__(self,n_vocab,embed_dim,hidden_size,lstm_dropout,output_size):
    super().__init__()

    self.V = n_vocab
    self.D = embed_dim
    self.M = hidden_size
    self.K = output_size

    self.embed = nn.Embedding(self.V,self.D)

    self.rnn = nn.LSTM(
        input_size = self.D,
        hidden_size=self.M,
        num_layers =1,
        batch_first=True,
        dropout=lstm_dropout
    )

    self.fc = nn.Linear(self.M,self.K)

  def forward(self,X):
    h0 = torch.zeros(1,X.size(0),self.M).to(device)
    c0 = torch.zeros(1,X.size(0),self.M).to(device)

    out = self.embed(X)
    out,_ = self.rnn(out,(h0,c0))
    out,_ = torch.max(out,1)
    out = self.fc(out)
    return out 

In [ ]:
len(wordtoidx)

14382

In [ ]:
model = RNN(14382,20,20,0.2,3)
model.to(device)

In [ ]:
#Later to restore:
model.load_state_dict(torch.load('./model_weights',map_location=torch.device("cpu")))
model.eval()

RNN(
  (embed): Embedding(14382, 20)
  (rnn): LSTM(20, 20, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=20, out_features=3, bias=True)
)

In [ ]:
input_test_data = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2, 3, 4, 5, 6, 7],
[295, 162, 715, 2776, 182, 1425, 513, 1426, 16, 3387, 144, 90, 2045, 917, 1426, 816, 68, 1550, 42, 858, 6]]

input_test_data = torch.from_numpy(np.array(input_test_data)).long()

In [ ]:
input_test_data

tensor([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    2,    3,    4,    5,    6,    7],
        [ 295,  162,  715, 2776,  182, 1425,  513, 1426,   16, 3387,  144,   90,
         2045,  917, 1426,  816,   68, 1550,   42,  858,    6]])

In [ ]:
len(input_test_data.shape)

2

In [ ]:
if len(input_test_data.shape) == 1:
  #single input
  model(input_test_data.view(1,-1)).argmax(dim=1)
elif (len(input_test_data.shape) == 2):
  #multiple input
  model(input_test_data).argmax(dim=1)
else:
  print(f'error in shapes: must be [N,D] got {input_test_data.shape}')

RuntimeError: ignored

#Getting Data

In [ ]:
pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aaassdfsdfef","key":"fb05b989359f0007deeb7a7b60aafcbc"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d olistbr/brazilian-ecommerce

 84% 36.0M/42.7M [00:00<00:00, 61.2MB/s]
100% 42.7M/42.7M [00:00<00:00, 108MB/s] 


In [ ]:
!unzip brazilian-ecommerce.zip

Archive:  brazilian-ecommerce.zip
  inflating: olist_customers_dataset.csv  
  inflating: olist_geolocation_dataset.csv  
  inflating: olist_order_items_dataset.csv  
  inflating: olist_order_payments_dataset.csv  
  inflating: olist_order_reviews_dataset.csv  
  inflating: olist_orders_dataset.csv  
  inflating: olist_products_dataset.csv  
  inflating: olist_sellers_dataset.csv  
  inflating: product_category_name_translation.csv  


#Custom transformers


In [ ]:
import pandas as pd

df = pd.read_csv("olist_order_reviews_dataset.csv")
df = df[["review_comment_title",'review_comment_message','review_score']]
df.head()

,review_comment_title,review_comment_message,review_score
0,NaN,NaN,4
1,NaN,NaN,5
2,NaN,NaN,5
3,NaN,Recebi bem antes do prazo estipulado.,5
4,NaN,Parabéns lojas lannister adorei comprar pela I...,5


In [ ]:
pip install unidecode

In [ ]:
target = df['review_score']

# 0 = positive ; 1 = neutral ; 2 = negative
target = target.map({1:2,2:2,3:1,4:0,5:0})
target = target.tolist()
len(target)

In [ ]:
import pandas as pd

df = pd.read_csv("olist_order_reviews_dataset.csv")
df = df[["review_comment_title",'review_comment_message','review_score']]

class AppendTitleWithMessage():
  def __init__(self):
    return None

  def fit(self,X,y=None):
    return self

  def transform(self,X):
    import numpy as np
    X = X.copy()

    X["review_comment"] = np.where(X["review_comment_title"].isnull()==False,X["review_comment_title"] +" "+ X["review_comment_message"],
                                X["review_comment_message"]) 

    X["review_comment"] = np.where(X["review_comment_message"].isnull(),
                                X["review_comment_title"], X["review_comment"] )
    return X


transformer = AppendTitleWithMessage()

df = transformer.transform(df)

df = df[df['review_comment'].isnull() == False]

test_df = df

class SelectAndRenameFeatures():
  def __init__(self):
    return None

  def fit(self,X,y=None):
    return self

  def transform(self,X):
    X = X.copy()

    X = X["review_comment"]
    X.rename("text",inplace=True)

    return X

transformer = SelectAndRenameFeatures()
df = transformer.transform(df)

class FormatText():
  def __init__(self):
    return None

  def fit(self,X,y=None):
    return self


  def format_input(self,text):
    import string
    import unidecode

    for punctuation in string.punctuation:
      text = text.replace(punctuation," ")
      text = text.lower()
      text = unidecode.unidecode(text)
    return text

  def transform(self,X):
    X = X.copy()
    X = X.apply(self.format_input)

    X = X.apply(lambda x: x.split())

    return X

transformer = FormatText()
df = transformer.transform(df)
# print(df)

class WordsToIndex():
  def __init__(self,index_file):
    self.index_file = index_file
    return None

  def fit(self,X,y=None):
    return self

  def load_dict(self):
    import pickle
    idxfile = open(self.index_file, "rb")
    output = pickle.load(idxfile)
    idxfile.close()

    return output

  def transform(self,X):
    X = X.copy()
    wordtoidx = self.load_dict()
 
    #word to int
    for i in range(len(X)):
      for j in range(len(X.iloc[i])):
        if X.iloc[i][j] in wordtoidx:
          X.iloc[i][j] = wordtoidx[X.iloc[i][j]]
        else:
          X.iloc[i][j] = 1
    
    return X

transformer = WordsToIndex(index_file='wordtoidx.pkl')
df = transformer.transform(df)


class ModelPredictor():
  def __init__(self,batch_size,device):
    self.batch_size = batch_size
    self.device = device

  def fit(self,X,y=None):
    return self

  def batches_generator(self,X):
    import numpy as np

    num_batches = int (np.ceil(len(X) / self.batch_size))

    batches = []
    for i in range(num_batches):
      x_batch = X[self.batch_size*i:self.batch_size*(i+1)]

      max_len = np.max([len(x) for x in x_batch])
      for j in range(len(x_batch)):
        x = x_batch[j]
        pad = [0] * (max_len - len(x))
        x_batch[j] = pad + x

      x_batch = torch.Tensor(x_batch).long()
      x_batch = x_batch.to(self.device)

      yield x_batch

  def load_model(self):
    model = RNN(14382,20,20,0.2,3)
    model.to(device)

    model.load_state_dict(torch.load('./model_weights',map_location=torch.device(self.device)))
    model.eval()
    return model

  def predict(self,X):
    X = X.copy()

    if isinstance(X,pd.Series) :
      X = X.tolist()

    model = self.load_model()

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    X_iter = lambda: self.batches_generator(X)
    
    outputs = []
    for inputs in X_iter():

      if len(inputs.shape) == 1:
        #single input
        output = model(inputs.view(1,-1)).argmax(dim=1)
      else:
        #multiple input
        output = model(inputs).argmax(dim=1)

        # print(f'error in shapes: must be [N,D] got {inputs.shape}')


      outputs.extend(output.tolist())

    return outputs

transformer = ModelPredictor(batch_size=32,device=device)
model_output = transformer.predict(df)
# print(len(model_output))
# print(model_output)

target = test_df['review_score']
# 0 = positive ; 1 = neutral ; 2 = negative
target = target.map({1:2,2:2,3:1,4:0,5:0})
target = target.tolist()

correct = 0
total = 0
for i in range(len(model_output)):
  if model_output[i] == target[i]:
    correct += 1

  total += 1

print('acc:',correct/total)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


acc: 0.8693022400073593


#Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

nlp_pipe = Pipeline([
    ("append texts",AppendTitleWithMessage()),
    ("select text feature",SelectAndRenameFeatures()),
    ("format text",FormatText()),
    ("transform words to int",WordsToIndex(index_file='wordtoidx.pkl')),
    ("model",ModelPredictor(batch_size=32,device=device)),
])

In [ ]:
df = pd.read_csv("olist_order_reviews_dataset.csv")
df = df[["review_comment_title",'review_comment_message','review_score']]
df = df[df['review_comment_message'].isnull() == False]


pipeline_output = nlp_pipe.predict(df)
print(pipeline_output)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


[0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 1, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 2, 2, 1, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 2, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 

In [ ]:
target = test_df['review_score']
# 0 = positive ; 1 = neutral ; 2 = negative
target = target.map({1:2,2:2,3:1,4:0,5:0})
target = target.tolist()

correct = 0
total = 0
for i in range(len(pipeline_output)):
  if pipeline_output[i] == target[i]:
    correct += 1

  total += 1

#acc i a lil different because i filtered the nan values 
print('acc:',correct/total)

acc: 0.8696141594615956
